In [34]:
import sys
import numpy as np
import torch
sys.path.append("../")  
import src.RBM as rbm


In [47]:
visible_dim = 327736 
hidden_dim = 1000 
sigma = 0.1
RBM =  rbm.GaussianBinaryRBM(visible_dim, hidden_dim, sigma)

In [48]:
print(RBM)

GaussianBinaryRBM()


In [49]:
v0 = np.random.uniform(-1, 1, visible_dim)
v0 = torch.tensor(v0, dtype=torch.float32)
h = RBM.sample_h(v0)

In [50]:
print(h)
print(len(h))
print(len(v0))

tensor([1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
        1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
        0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
        0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
        0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
        0., 1., 0., 0., 1., 1., 1., 1., 

In [51]:
RBM.contrastive_divergence(v0)

RuntimeError: inconsistent tensor size, expected tensor [1000] and src [327736] to have the same number of elements, but got 1000 and 327736 elements respectively